In [1]:
import sys
import struct
import os
from scapy.all import *
from matplotlib import pyplot as plt
from prettytable import PrettyTable
from utils import *
import numpy as np
import matplotlib.pyplot as plt

In [13]:
def get_connections(filename):
    Connections = []
    my_ip = "172.16.105.110"
        
    a = rdpcap(filename)
    sessions = a.sessions()
    
    for session in sessions:
        for id_p,packet in enumerate(sessions[session]):
            if(packet.haslayer(ARP) or packet.haslayer(DNS)):
                continue
                 
            if(packet.haslayer(IP)):
                if(packet.haslayer(TCP)):                
                    flag = False
                    idx = len(Connections)
                    for i,con in enumerate(Connections):
                        if match_TCP(con, packet):
                            flag = True
                            idx = i
                            break
                    if not flag:
                        if packet[IP].src==my_ip:
                            Connections.append({
                                's_ip':packet[IP].src,'d_ip':packet[IP].dst,
                                's_port':packet[TCP].sport,'d_port':packet[TCP].dport,
                                'type':"TCP",
                                'snd_packets':[],
                                'rcv_packets':[]
                            })
                        else:
                            Connections.append({
                                's_ip':packet[IP].dst,'d_ip':packet[IP].src,
                                's_port':packet[TCP].dport,'d_port':packet[TCP].sport,
                                'type':"TCP",
                                'snd_packets':[],
                                'rcv_packets':[]
                            })
                        
                    if packet[IP].src==my_ip:
                        Connections[idx]['snd_packets'].append({
                            "seq":packet[TCP].seq, "len":len(packet[TCP].payload),
                            "flag":packet[TCP].flags, "order": id_p
                        })
                    else:
                        Connections[idx]['rcv_packets'].append({
                            "seq":packet[TCP].seq, "len":len(packet[TCP].payload),
                            "flag":packet[TCP].flags, "order": id_p
                        })
                    pass
            
                elif(packet.haslayer(UDP)):
                    flag = False
                    idx = len(Connections)
                    for i,con in enumerate(Connections):
                        if match_UDP(con, packet):
                            flag = True
                            idx=i
                            break
                    if not flag:
                        if packet[IP].src==my_ip:
                            Connections.append({
                                's_ip':packet[IP].src,'d_ip':packet[IP].dst,
                                's_port':packet[UDP].sport,'d_port':packet[UDP].dport,
                                'type':"UDP",
                                'snd_packets':[],
                                'rcv_packets':[]
                            })
                        else:
                            Connections.append({
                                's_ip':packet[IP].dst,'d_ip':packet[IP].src,
                                's_port':packet[UDP].dport,'d_port':packet[UDP].sport,
                                'type':"UDP",
                                'snd_packets':[],
                                'rcv_packets':[]
                            })
                        
                    if packet[IP].src==my_ip:
                        Connections[idx]['snd_packets'].append({
                            "len":len(packet[UDP].payload),
                                "order": id_p
                        })
                    else:
                        Connections[idx]['rcv_packets'].append({
                            "len":len(packet[UDP].payload),
                            "order": id_p
                        })
                    pass
                else:
                    pass
    #                 break
    
    x = PrettyTable()
    
    x.field_names = ["Connection", "Type", "Num Pkt H2R", "Num Pkt R2H", "Len Data H2R", "Len Data R2H"]
    
    for con in Connections:
        x.add_row([f"{con['s_ip']}:{con['s_port']}->{con['d_ip']}:{con['d_port']}", 
                   con['type'], 
                   len(con['snd_packets']), len(con['rcv_packets']), 
                   sum([pkt['len'] for pkt in con['snd_packets']]), 
                   sum([pkt['len'] for pkt in con['rcv_packets']])])
        
    
    print(x)

In [14]:
get_connections("zoom/zoom_full.pcapng")

+-------------------------------------------+------+-------------+-------------+--------------+--------------+
|                 Connection                | Type | Num Pkt H2R | Num Pkt R2H | Len Data H2R | Len Data R2H |
+-------------------------------------------+------+-------------+-------------+--------------+--------------+
|  172.16.105.110:60450->170.114.10.89:443  | TCP  |      2      |      0      |      31      |      0       |
|  172.16.105.110:60452->170.114.10.89:443  | TCP  |      2      |      0      |      31      |      0       |
|  172.16.105.110:60454->170.114.10.89:443  | TCP  |      2      |      0      |      31      |      0       |
|  172.16.105.110:56786->170.114.10.83:443  | TCP  |      2      |      0      |      31      |      0       |
|  172.16.105.110:52210->170.114.10.71:443  | TCP  |      11     |      9      |     3015     |    34949     |
| 172.16.105.110:45782->198.251.148.137:443 | TCP  |      8      |      6      |     517      |     4871     |
|

In [22]:
get_connections("zoom/zoom_both.pcapng")

+-------------------------------------------+------+-------------+-------------+--------------+--------------+
|                 Connection                | Type | Num Pkt H2R | Num Pkt R2H | Len Data H2R | Len Data R2H |
+-------------------------------------------+------+-------------+-------------+--------------+--------------+
| 172.16.105.110:48778->198.251.131.69:8801 | UDP  |     555     |      15     |    573070    |     1178     |
| 172.16.105.110:41758->198.251.131.69:8801 | UDP  |      6      |      2      |     418      |     162      |
|  172.16.105.110:42166->198.251.131.69:443 | TCP  |      3      |      4      |      90      |      90      |
| 172.16.105.110:41800->198.251.131.69:8801 | UDP  |      2      |      2      |     162      |     162      |
+-------------------------------------------+------+-------------+-------------+--------------+--------------+


In [16]:
get_connections("zoom/zoom_noaudio.pcapng")

+-------------------------------------------+------+-------------+-------------+--------------+--------------+
|                 Connection                | Type | Num Pkt H2R | Num Pkt R2H | Len Data H2R | Len Data R2H |
+-------------------------------------------+------+-------------+-------------+--------------+--------------+
|  172.16.105.110:60452->170.114.10.89:443  | TCP  |      2      |      3      |      0       |      43      |
|  172.16.105.110:60454->170.114.10.89:443  | TCP  |      2      |      2      |      0       |      37      |
|  172.16.105.110:42166->198.251.131.69:443 | TCP  |      8      |      10     |     703      |     230      |
| 172.16.105.110:48778->198.251.131.69:8801 | UDP  |      4      |      4      |     324      |     324      |
|  172.16.105.110:56786->170.114.10.83:443  | TCP  |      2      |      3      |      0       |      43      |
| 172.16.105.110:41758->198.251.131.69:8801 | UDP  |      4      |      4      |     324      |     324      |
|

In [17]:
get_connections("zoom/zoom_novideo.pcapng")

+-------------------------------------------+------+-------------+-------------+--------------+--------------+
|                 Connection                | Type | Num Pkt H2R | Num Pkt R2H | Len Data H2R | Len Data R2H |
+-------------------------------------------+------+-------------+-------------+--------------+--------------+
|  172.16.105.110:42166->198.251.131.69:443 | TCP  |      10     |      10     |     308      |     266      |
| 172.16.105.110:48778->198.251.131.69:8801 | UDP  |      4      |      4      |     324      |     324      |
| 172.16.105.110:41758->198.251.131.69:8801 | UDP  |      10     |      4      |     708      |     324      |
| 172.16.105.110:41800->198.251.131.69:8801 | UDP  |      4      |      4      |     324      |     324      |
|  255.255.255.255:5678->172.16.105.1:5678  | UDP  |      0      |      1      |      0       |     119      |
+-------------------------------------------+------+-------------+-------------+--------------+--------------+


In [18]:
get_connections("zoom/zoom_none.pcapng")

+-------------------------------------------+------+-------------+-------------+--------------+--------------+
|                 Connection                | Type | Num Pkt H2R | Num Pkt R2H | Len Data H2R | Len Data R2H |
+-------------------------------------------+------+-------------+-------------+--------------+--------------+
|  172.16.105.110:42166->198.251.131.69:443 | TCP  |      10     |      11     |     308      |     266      |
| 172.16.105.110:48778->198.251.131.69:8801 | UDP  |      4      |      4      |     324      |     324      |
| 172.16.105.110:41758->198.251.131.69:8801 | UDP  |      4      |      4      |     324      |     324      |
| 172.16.105.110:41800->198.251.131.69:8801 | UDP  |      4      |      4      |     324      |     324      |
+-------------------------------------------+------+-------------+-------------+--------------+--------------+


In [23]:
get_connections("zoom/zoom_turn.pcapng")

+--------------------------------------------+------+-------------+-------------+--------------+--------------+
|                 Connection                 | Type | Num Pkt H2R | Num Pkt R2H | Len Data H2R | Len Data R2H |
+--------------------------------------------+------+-------------+-------------+--------------+--------------+
|  172.16.105.110:52212->170.114.10.71:443   | TCP  |      12     |      10     |     3017     |    35222     |
|       255.255.255.255:67->0.0.0.0:68       | UDP  |      0      |      1      |      0       |     300      |
|  172.16.105.110:45500->198.251.151.22:443  | TCP  |      8      |      6      |     517      |     4871     |
| 172.16.105.110:53684->198.251.151.134:443  | TCP  |      8      |      6      |     517      |     4871     |
| 172.16.105.110:32900->193.123.147.171:443  | TCP  |      14     |      10     |     1874     |     5625     |
| 172.16.105.110:49180->193.123.139.126:443  | TCP  |      15     |      10     |     828      |     502